# ETL para o banco de dados Neo4j

## Importações

In [1]:
import os
import sys

from pyspark.sql import SparkSession

## Constantes

In [2]:
DATASET_PATH = '../../dataset'

USER_FILTERED_PATH = os.path.join(DATASET_PATH, 'user-filtered.csv')

## Código para resolver um problema de versão encontrada pelo PySpark

A resolução do erro foi encontrada em uma resposta no [StackOverflow](https://stackoverflow.com/a/65010346)

In [3]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Configurando inicialização do "conector" Neo4j spark

In [4]:
spark = SparkSession.\
        builder.\
        config("spark.jars", "../../spark-neo4j/neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar").\
        getOrCreate()

## Lendo o arquivo user-filtered.csv

In [5]:
df_user_filtred = spark.read.csv(USER_FILTERED_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

In [6]:
df_user_filtred.show()

+-------+--------+------+
|user_id|anime_id|rating|
+-------+--------+------+
|      0|      67|     9|
|      0|    6702|     7|
|      0|     242|    10|
|      0|    4898|     0|
|      0|      21|    10|
|      0|      24|     9|
|      0|    2104|     0|
|      0|    4722|     8|
|      0|    6098|     6|
|      0|    3125|     9|
|      0|     481|    10|
|      0|      68|     6|
|      0|    1689|     6|
|      0|    2913|     6|
|      0|    1250|     7|
|      0|     356|     9|
|      0|     121|     9|
|      0|     430|     9|
|      0|    1829|     7|
|      0|    1571|    10|
+-------+--------+------+
only showing top 20 rows



## Salvando os dados no banco de dados

### Salvando os nós de usuários

In [7]:
display(df_user_filtred.select('user_id').distinct().show())

+-------+
|user_id|
+-------+
|    148|
|    463|
|    471|
|    496|
|    833|
|   1088|
|   1342|
|   1580|
|   1591|
|   1645|
|   1829|
|   1959|
|   2122|
|   2142|
|   2366|
|   2659|
|   2866|
|   3175|
|   3749|
|   3794|
+-------+
only showing top 20 rows



None

In [8]:
df_user_filtred.select('user_id').distinct().write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":User") \
  .save()

### Salvando os nós de animes

In [9]:
display(df_user_filtred.select('anime_id').distinct().show())

+--------+
|anime_id|
+--------+
|    1829|
|    3918|
|    5300|
|    6336|
|    8086|
|   30654|
|    2142|
|   36525|
|   17389|
|   22097|
|   38422|
|     496|
|   36538|
|   25517|
|    1088|
|     463|
|    9465|
|   30903|
|   11033|
|   37307|
+--------+
only showing top 20 rows



None

In [10]:
df_user_filtred.select('anime_id').distinct().write \
  .format("org.neo4j.spark.DataSource") \
  .mode("Append") \
  .option("url", "bolt://localhost:7687") \
  .option("labels", ":Anime") \
  .save()

### Salvando as relações entre usuários e animes

In [ ]:
df_user_filtred.write.format("org.neo4j.spark.DataSource")\
        .mode("Overwrite")\
        .option("url", "neo4j://localhost:7687")\
        .option("relationship", "Rating")\
        .option("relationship.save.strategy", "keys")\
        .option("relationship.source.labels", ":User")\
        .option("relationship.source.save.mode", "overwrite")\
        .option("relationship.source.node.keys", "user_id:user_id")\
        .option("relationship.target.labels", ":Anime")\
        .option("relationship.target.save.mode", "overwrite")\
        .option("relationship.target.node.keys", "anime_id:anime_id")\
        .option("relationship.properties", "rating")\
        .save()